In [1]:
import pandas as pd
import numpy as np
from ana_tool import category_ana as ca

In [5]:
df_all = pd.read_excel(r"C:\Users\Administrator\Downloads\tb_OTC_眼药水_all.xlsx",sheet_name="Sheet1")

In [6]:
df = ca.init_category_df(gap_num=10,wps_df=df_all,obj_to_date="时间",top_limit=500)

In [7]:
'''if CONTAINS([症状],'结膜炎') then '结膜炎'
ELSEIF CONTAINS([症状],'白内障') then '白内障'
ELSEIF CONTAINS([症状],'异物感') then '异物感'
ELSEIF CONTAINS([症状],'视力下降') then '视力下降'
ELSEIF CONTAINS([症状],'近视') then '近视'
ELSEIF CONTAINS([症状],'干') then '眼部干涩'
ELSEIF CONTAINS([症状],'疲劳') then '眼疲劳'

else 'その他' END'''

def zhengzhuangenlei(j):
    k = str(j["症状"])
    m = str(j["适应症"])
    if "结膜炎" in k or "结膜炎" in m:
        zz = "结膜炎"
    elif "白内障" in k or "白内障" in m:
        zz = "白内障"
    elif "异物感" in k or "异物感" in m:
        zz = "异物感"
    elif "视力下降" in k or "视力下降" in m:
        zz = "视力下降"
    elif "近视" in k or "近视" in m:
        zz = "近视"
    elif "干" in k or "干" in m:
        zz = "眼部干涩"
    elif "疲劳" in k or "疲劳" in m:
        zz = "眼疲劳"
    else:
        zz = "その他"
    return zz

In [9]:
df["症状分类"] = df.apply(lambda x:zhengzhuangenlei(x),axis=1)

In [10]:
def find_num(s):
    s = str(s)
    end_num = s.find(",")
    return int(s[1:end_num])

In [11]:
df["价格辅助"] = df["价格带"].apply(lambda c:find_num(c)).astype(int)

In [12]:
for pivot_target in ["销售额","销量"]:
    for brand in ['参天','海露','珍视明','瑞珠','rohto','莎普爱思','润怡',"mentholatum/曼秀雷敦",'大正制药',"lion/狮王",'小林制药']:
            df_toana = df[df["品牌"].isin([brand])]
            df_priceana = ca.salesPivotPercent(df = df_toana,index="价格带",value = pivot_target,column="年",agg_func_list=[2021,2022,2023])
            df_priceana["price_sub"]=df_priceana.index.to_series().apply(lambda y:find_num(y)).astype(int)
            df_priceana = df_priceana.sort_values(by="price_sub",ascending=True)
            df_salesana = ca.salesPivotPercent(df=df_toana,index = "品牌",value=pivot_target,column="年",agg_func_list=[2021,2022,2023])
            df_zzana = ca.salesPivotPercent(df = df_toana,index="症状分类",value = pivot_target,column="年",agg_func_list=[2021,2022,2023])
            df_dpana = ca.salesPivotPercent(df = df_toana,index="店铺名",value = pivot_target,column="年",agg_func_list=[2021,2022,2023])
            if brand == "mentholatum/曼秀雷敦":
                brand ="曼秀雷敦"
            elif brand =="lion/狮王":
                brand = "狮王"
            else:
                brand = brand
            with pd.ExcelWriter("D:/lion240401/数量调查"+"/tb_"+brand+pivot_target+".xlsx",engine="openpyxl") as new_wb:
                df_salesana.to_excel(new_wb,sheet_name = pivot_target)
                df_zzana.to_excel(new_wb,sheet_name = "症状分类")
                df_priceana.to_excel(new_wb,sheet_name = "价格带")
                df_dpana.to_excel(new_wb,sheet_name = "店铺")


d:\scrips\ana_tool\category_ana.py:76: FutureWarning: The provided callable <function sum at 0x000002162A4284A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(df,index=index,values=value,columns=column,aggfunc=[np.sum])
d:\scrips\ana_tool\category_ana.py:76: FutureWarning: The provided callable <function sum at 0x000002162A4284A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(df,index=index,values=value,columns=column,aggfunc=[np.sum])
d:\scrips\ana_tool\category_ana.py:76: FutureWarning: The provided callable <function sum at 0x000002162A4284A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep curren